<a href="https://colab.research.google.com/github/umkiyoung/Time_series_analysis/blob/main/FINRL_Application_KY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FINRL Application

- ETF 유니버스 데이터 사용
- 예측 데이터 어떻게 사용할지 -> 예측 결과를 indicator로 줄 수 있을 듯. 실험 필요함.
- 포트폴리오 구성 방법 논의 필요
- 최고의 Baseline?

### Package Install

In [1]:
## install required packages
!pip install swig
!pip install wrds
!pip install pyportfolioopt
## install finrl library
!apt-get update -y -qq && apt-get install -y -qq cmake libopenmpi-dev python3-dev zlib1g-dev libgl1-mesa-glx swig
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

'apt-get'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


  Cloning https://github.com/AI4Finance-Foundation/FinRL.git to c:\users\public\documents\estsoft\creatortemp\pip-req-build-smzy9syl
  Resolved https://github.com/AI4Finance-Foundation/FinRL.git to commit d9f0af8021e0995dec71043ed517af954e3f1a4d
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to c:\users\public\documents\estsoft\creatortemp\pip-install-v1rhdnmn\elegantrl_1853f2277c0b49eb82c7307f78e834fc
  Resolved https://github.com/AI4Finance-Foundation/ElegantRL.git to commit 0c019eec035391dbe7aca1464ed6a0067e5a130f
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/FinRL.git 'C:\Users\Public\Documents\ESTsoft\CreatorTemp\pip-req-build-smzy9syl'
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/ElegantRL.git 'C:\Users\Public\Documents\ESTsoft\CreatorTemp\pip-install-v1rhdnmn\elegantrl_1853f2277c0b49eb82c7307f78e834fc'


In [2]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from stable_baselines3.common.logger import configure
from finrl.meta.data_processor import DataProcessor

from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from pprint import pprint

import sys
sys.path.append("../FinRL")

import itertools

c:\Users\se99a\anaconda3\lib\site-packages\pyfolio\pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


### Create Folders

In [200]:
from finrl import config
from finrl import config_tickers
import os
from finrl.main import check_and_make_directories
from finrl.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
    TRAIN_START_DATE,
    TRAIN_END_DATE,
    TEST_START_DATE,
    TEST_END_DATE,
    TRADE_START_DATE,
    TRADE_END_DATE,
)
check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])



In [201]:
# from config.py, TRAIN_START_DATE is a string
TRAIN_START_DATE
# from config.py, TRAIN_END_DATE is a string
TRAIN_END_DATE

'2020-07-31'

In [202]:
TRAIN_START_DATE = '2010-01-01'
TRAIN_END_DATE = '2021-10-01'
TRADE_START_DATE = '2021-10-01'
TRADE_END_DATE = '2023-05-01'

In [203]:
tickers = ['XLB','XLE','XLF','XLI','XLK','XLP','XLU','XLV','XLY']

In [204]:
df = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TRADE_END_DATE,
                     ticker_list = tickers).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (30177, 8)


In [205]:
df

,date,open,high,low,close,volume,tic,day
0,2010-01-04,33.580002,34.020000,33.450001,25.488928,7567500,XLB,0
1,2010-01-04,57.919998,58.810001,57.790001,38.080170,16928400,XLE,0
2,2010-01-04,11.795288,11.965881,11.770918,9.385942,92942347,XLF,0
3,2010-01-04,28.090000,28.320000,27.959999,21.880194,7471500,XLI,0
4,2010-01-04,23.139999,23.290001,23.100000,19.150225,8449400,XLK,0
...,...,...,...,...,...,...,...,...
30172,2023-04-28,149.210007,150.860001,149.050003,150.830002,5893000,XLK,4
30173,2023-04-28,77.230003,77.580002,77.040001,77.440002,9757600,XLP,4
30174,2023-04-28,68.879997,69.389999,68.550003,68.980003,12123400,XLU,4
30175,2023-04-28,132.240005,133.910004,132.029999,133.529999,7917300,XLV,4


In [206]:
df.sort_values(['date','tic'],ignore_index=True).head()

,date,open,high,low,close,volume,tic,day
0,2010-01-04,33.580002,34.020000,33.450001,25.488928,7567500,XLB,0
1,2010-01-04,57.919998,58.810001,57.790001,38.080170,16928400,XLE,0
2,2010-01-04,11.795288,11.965881,11.770918,9.385942,92942347,XLF,0
3,2010-01-04,28.090000,28.320000,27.959999,21.880194,7471500,XLI,0
4,2010-01-04,23.139999,23.290001,23.100000,19.150225,8449400,XLK,0


### ETF 도메인 별로 최적의 하이퍼 파라미터 튜닝을 진행해보자

In [237]:
INDICATORS

['macd',
 'boll_ub',
 'boll_lb',
 'rsi_30',
 'cci_30',
 'dx_30',
 'close_30_sma',
 'close_60_sma']

INDICATORS는 FINRL 기본 베이스라인으로 해도 괜찮을 것 같음

https://github.com/jealous/stockstats


- change (in percent)
- delta
- permutation (zero-based)
- log return
- max in range
- min in range
- middle = (close + high + low) / 3
- compare: le, ge, lt, gt, eq, ne
- count: both backward(c) and forward(fc)
- cross: including upward cross and downward cross
- SMA: Simple Moving Average
- EMA: Exponential Moving Average
- MSTD: Moving Standard Deviation
- MVAR: Moving Variance
- RSV: Raw Stochastic Value
- RSI: Relative Strength Index
- KDJ: Stochastic Oscillator
- Bolling: Bollinger Band
- MACD: Moving Average Convergence Divergence
- CR: Energy Index (Intermediate Willingness Index)
- WR: Williams Overbought/Oversold index
- CCI: Commodity Channel Index
- TR: True Range
- ATR: Average True Range
- DMA: Different of Moving Average (10, 50)
- DMI: Directional Moving Index, including
- +DI: Positive Directional Indicator
- -DI: Negative Directional Indicator
- ADX: Average Directional Movement Index
- ADXR: Smoothed Moving Average of ADX
- TRIX: Triple Exponential Moving Average
- TEMA: Another Triple Exponential Moving Average
- VR: Volume Variation Index
- MFI: Money Flow Index
- VWMA: Volume Weighted Moving Average
- CHOP: Choppiness Index
- KAMA: Kaufman's Adaptive Moving Average
- PPO: Percentage Price Oscillator
- StochRSI: Stochastic RSI
- WT: LazyBear's Wave Trend
- Supertrend: with the Upper Band and Lower Band

In [238]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = INDICATORS,
                    use_vix=True,
                    use_turbulence=True,
                    user_defined_feature = False)

processed = fe.preprocess_data(df)

Successfully added technical indicators
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (3352, 8)
Successfully added vix
Successfully added turbulence index


In [239]:
processed

,date,open,high,low,close,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2010-01-04,33.580002,34.020000,33.450001,25.488928,7567500,XLB,0,0.000000,25.646683,25.413585,100.000000,66.666667,100.000000,25.488928,25.488928,20.040001,0.000000
1,2010-01-04,57.919998,58.810001,57.790001,38.080170,16928400,XLE,0,0.000000,25.646683,25.413585,100.000000,66.666667,100.000000,38.080170,38.080170,20.040001,0.000000
2,2010-01-04,11.795288,11.965881,11.770918,9.385942,92942347,XLF,0,0.000000,25.646683,25.413585,100.000000,66.666667,100.000000,9.385942,9.385942,20.040001,0.000000
3,2010-01-04,28.090000,28.320000,27.959999,21.880194,7471500,XLI,0,0.000000,25.646683,25.413585,100.000000,66.666667,100.000000,21.880194,21.880194,20.040001,0.000000
4,2010-01-04,23.139999,23.290001,23.100000,19.150225,8449400,XLK,0,0.000000,25.646683,25.413585,100.000000,66.666667,100.000000,19.150225,19.150225,20.040001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30163,2023-04-27,146.800003,149.320007,146.440002,149.210007,6042400,XLK,3,0.906210,151.760708,145.221293,57.138356,43.554032,10.029523,147.182253,143.204452,17.030001,4.575693
30164,2023-04-27,76.220001,77.110001,76.169998,77.029999,11287800,XLP,3,0.892918,77.176238,74.163761,61.967635,105.218450,34.078385,74.680582,73.532967,17.030001,4.575693
30165,2023-04-27,68.459999,69.139999,68.400002,69.080002,12466700,XLU,3,0.633085,70.845444,67.185557,53.290458,42.102205,4.528401,67.964842,67.168816,17.030001,4.575693
30166,2023-04-27,131.770004,132.589996,130.619995,132.399994,10008900,XLV,3,1.086855,136.802596,129.195405,52.366155,21.773054,4.270448,130.932522,130.043549,17.030001,4.575693


In [240]:
prediction_dataset = pd.read_csv("DLinear_prediction.csv")
prediction_dataset


,Unnamed: 0,date,XLB,XLE,XLF,XLI,XLK,XLP,XLU,XLV,XLY
0,72,2010-04-19,26.115305,38.589978,9.725630,25.475499,18.537268,19.907799,18.991999,25.336764,27.513101
1,73,2010-04-20,26.130648,38.601034,9.730219,25.529918,18.557873,19.916413,18.979760,25.338780,27.579060
2,74,2010-04-21,26.142984,38.697584,9.749220,25.599275,18.580593,19.937981,18.980213,25.345207,27.665377
3,75,2010-04-22,26.161770,38.794925,9.767099,25.671408,18.609039,19.963366,18.989870,25.325928,27.749684
4,76,2010-04-23,26.202324,38.914966,9.794323,25.753211,18.638544,19.983188,18.998598,25.290620,27.862330
...,...,...,...,...,...,...,...,...,...,...,...
3278,392,2023-04-25,79.455027,83.468285,31.909410,97.957891,147.103831,73.961955,69.113930,132.733971,146.176733
3279,393,2023-04-26,79.330117,83.542106,31.851491,97.896812,147.270316,74.095982,69.123147,132.804389,145.899568
3280,394,2023-04-27,79.133475,83.431870,31.761901,97.659214,147.305152,74.186167,69.092036,132.738247,145.628134
3281,395,2023-04-28,78.971304,83.409488,31.672824,97.517751,147.356683,74.262698,69.115708,132.711250,145.692628


In [241]:
#transform the dataframe to date, tic, close format
prediction_dataset = pd.melt(prediction_dataset, id_vars=['date'], value_vars=tickers)
prediction_dataset = prediction_dataset.rename(columns={'variable': 'tic', 'value': 'prediction'})

#transform the dataframe ordered by date
prediction_dataset = prediction_dataset.sort_values(['date','tic'],ignore_index=True)

In [242]:
prediction_dataset

,date,tic,prediction
0,2010-04-19,XLB,26.115305
1,2010-04-19,XLE,38.589978
2,2010-04-19,XLF,9.725630
3,2010-04-19,XLI,25.475499
4,2010-04-19,XLK,18.537268
...,...,...,...
29542,2023-05-01,XLK,147.578356
29543,2023-05-01,XLP,74.377381
29544,2023-05-01,XLU,69.131218
29545,2023-05-01,XLV,132.781385


In [243]:
data = processed[['date','tic']]


def erase_same_rows(dataframe):
    dataframe = dataframe.drop_duplicates(subset=['date','tic'], keep='last')
    return dataframe

erased_dt = erase_same_rows(pd.merge(data, prediction_dataset, on=['date','tic'], how='left')
                .sort_values(['date','tic'],ignore_index=True))

erased_dt

,date,tic,prediction
0,2010-01-04,XLB,NaN
1,2010-01-04,XLE,NaN
2,2010-01-04,XLF,NaN
3,2010-01-04,XLI,NaN
4,2010-01-04,XLK,NaN
...,...,...,...
30172,2023-04-27,XLK,147.305152
30173,2023-04-27,XLP,74.186167
30174,2023-04-27,XLU,69.092036
30175,2023-04-27,XLV,132.738247


In [244]:
processed = pd.merge(processed, erased_dt, on=['date','tic'], how='left')
processed

,date,open,high,low,close,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence,prediction
0,2010-01-04,33.580002,34.020000,33.450001,25.488928,7567500,XLB,0,0.000000,25.646683,25.413585,100.000000,66.666667,100.000000,25.488928,25.488928,20.040001,0.000000,NaN
1,2010-01-04,57.919998,58.810001,57.790001,38.080170,16928400,XLE,0,0.000000,25.646683,25.413585,100.000000,66.666667,100.000000,38.080170,38.080170,20.040001,0.000000,NaN
2,2010-01-04,11.795288,11.965881,11.770918,9.385942,92942347,XLF,0,0.000000,25.646683,25.413585,100.000000,66.666667,100.000000,9.385942,9.385942,20.040001,0.000000,NaN
3,2010-01-04,28.090000,28.320000,27.959999,21.880194,7471500,XLI,0,0.000000,25.646683,25.413585,100.000000,66.666667,100.000000,21.880194,21.880194,20.040001,0.000000,NaN
4,2010-01-04,23.139999,23.290001,23.100000,19.150225,8449400,XLK,0,0.000000,25.646683,25.413585,100.000000,66.666667,100.000000,19.150225,19.150225,20.040001,0.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30163,2023-04-27,146.800003,149.320007,146.440002,149.210007,6042400,XLK,3,0.906210,151.760708,145.221293,57.138356,43.554032,10.029523,147.182253,143.204452,17.030001,4.575693,147.305152
30164,2023-04-27,76.220001,77.110001,76.169998,77.029999,11287800,XLP,3,0.892918,77.176238,74.163761,61.967635,105.218450,34.078385,74.680582,73.532967,17.030001,4.575693,74.186167
30165,2023-04-27,68.459999,69.139999,68.400002,69.080002,12466700,XLU,3,0.633085,70.845444,67.185557,53.290458,42.102205,4.528401,67.964842,67.168816,17.030001,4.575693,69.092036
30166,2023-04-27,131.770004,132.589996,130.619995,132.399994,10008900,XLV,3,1.086855,136.802596,129.195405,52.366155,21.773054,4.270448,130.932522,130.043549,17.030001,4.575693,132.738247


In [280]:
list_ticker = processed["tic"].unique().tolist() #ticker 리스트 불러오기
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str)) #전체 데이터 날짜 날짜 리스트
combination = list(itertools.product(list_date,list_ticker)) #date, ticker 의 combination

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left") #date, tic combination 기준으로 우측에 해당되는 정보들 정리
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)
processed_full

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence,prediction
0,2010-01-04,XLB,33.580002,34.020000,33.450001,25.488928,7567500.0,0.0,0.000000,25.646683,25.413585,100.000000,66.666667,100.000000,25.488928,25.488928,20.040001,0.000000,0.000000
1,2010-01-04,XLE,57.919998,58.810001,57.790001,38.080170,16928400.0,0.0,0.000000,25.646683,25.413585,100.000000,66.666667,100.000000,38.080170,38.080170,20.040001,0.000000,0.000000
2,2010-01-04,XLF,11.795288,11.965881,11.770918,9.385942,92942347.0,0.0,0.000000,25.646683,25.413585,100.000000,66.666667,100.000000,9.385942,9.385942,20.040001,0.000000,0.000000
3,2010-01-04,XLI,28.090000,28.320000,27.959999,21.880194,7471500.0,0.0,0.000000,25.646683,25.413585,100.000000,66.666667,100.000000,21.880194,21.880194,20.040001,0.000000,0.000000
4,2010-01-04,XLK,23.139999,23.290001,23.100000,19.150225,8449400.0,0.0,0.000000,25.646683,25.413585,100.000000,66.666667,100.000000,19.150225,19.150225,20.040001,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43753,2023-04-27,XLK,146.800003,149.320007,146.440002,149.210007,6042400.0,3.0,0.906210,151.760708,145.221293,57.138356,43.554032,10.029523,147.182253,143.204452,17.030001,4.575693,147.305152
43754,2023-04-27,XLP,76.220001,77.110001,76.169998,77.029999,11287800.0,3.0,0.892918,77.176238,74.163761,61.967635,105.218450,34.078385,74.680582,73.532967,17.030001,4.575693,74.186167
43755,2023-04-27,XLU,68.459999,69.139999,68.400002,69.080002,12466700.0,3.0,0.633085,70.845444,67.185557,53.290458,42.102205,4.528401,67.964842,67.168816,17.030001,4.575693,69.092036
43756,2023-04-27,XLV,131.770004,132.589996,130.619995,132.399994,10008900.0,3.0,1.086855,136.802596,129.195405,52.366155,21.773054,4.270448,130.932522,130.043549,17.030001,4.575693,132.738247


In [288]:
#Indicator 재조정

INDICATORS = ['macd',
 'boll_ub',
 'boll_lb',
 'rsi_30',
 'cci_30',
 'dx_30',
 'close_30_sma',
 'close_60_sma'] + ['prediction']

INDICATORS

['macd',
 'boll_ub',
 'boll_lb',
 'rsi_30',
 'cci_30',
 'dx_30',
 'close_30_sma',
 'close_60_sma',
 'prediction']

### Data split

전체를 다 학습하면 안되는거 아닌가? 

In [289]:
train = data_split(processed_full, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(processed_full, TRADE_START_DATE,TRADE_END_DATE)
print(len(train))
print(len(trade))

26613
3555


In [290]:
len(train)+len(trade)

30168

### environment setting

In [291]:
stock_dimension = len(train.tic.unique()) 
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension #현재 상태를 나타내는 state를 정의. 1+ ticker 개수 x2 + 인디케이터 개수 x ticker개수 
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 9, State Space: 100


In [292]:
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}


e_train_gym = StockTradingEnv(df = train, **env_kwargs)

In [293]:
print(buy_cost_list)
num_stock_shares

[0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001]


[0, 0, 0, 0, 0, 0, 0, 0, 0]

In [294]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


Training Part using A2C

In [295]:
agent = DRLAgent(env = env_train)

if_using_a2c = True ##a2c만 사용해보자
if_using_ddpg = False
if_using_ppo = False
if_using_td3 = False
if_using_sac = False

In [296]:
agent = DRLAgent(env = env_train)
model_a2c = agent.get_model("a2c")

if if_using_a2c:
  # set up logger
  tmp_path = RESULTS_DIR + '/a2c'
  new_logger_a2c = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_a2c.set_logger(new_logger_a2c)


{'n_steps': 5, 'ent_coef': 0.01, 'learning_rate': 0.0007}
Using cpu device
Logging to results/a2c


In [297]:
trained_a2c = agent.train_model(model=model_a2c, 
                             tb_log_name='a2c',
                             total_timesteps=50000) if if_using_a2c else None

--------------------------------------
| time/                 |            |
|    fps                | 218        |
|    iterations         | 100        |
|    time_elapsed       | 2          |
|    total_timesteps    | 500        |
| train/                |            |
|    entropy_loss       | -12.8      |
|    explained_variance | -0.1       |
|    learning_rate      | 0.0007     |
|    n_updates          | 99         |
|    policy_loss        | -17.4      |
|    reward             | 0.18696393 |
|    std                | 1.01       |
|    value_loss         | 2.91       |
--------------------------------------
--------------------------------------
| time/                 |            |
|    fps                | 218        |
|    iterations         | 200        |
|    time_elapsed       | 4          |
|    total_timesteps    | 1000       |
| train/                |            |
|    entropy_loss       | -12.8      |
|    explained_variance | -0.00791   |
|    learning_rate      |

## In-sample Performance

Assume that the initial capital is $1,000,000.

### Set turbulence threshold
Set the turbulence threshold to be greater than the maximum of insample turbulence data. If current turbulence index is greater than the threshold, then we assume that the current market is volatile

주식시장 변동성지수를 이용해서 언제 위험한 상황인지를 나타낸다.

In [298]:
data_risk_indicator = processed_full[(processed_full.date<TRAIN_END_DATE) & (processed_full.date>=TRAIN_START_DATE)]
insample_risk_indicator = data_risk_indicator.drop_duplicates(subset=['date'])

In [299]:
insample_risk_indicator

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence,prediction
0,2010-01-04,XLB,33.580002,34.020000,33.450001,25.488928,7567500.0,0.0,0.000000,25.646683,25.413585,100.000000,66.666667,100.000000,25.488928,25.488928,20.040001,0.000000,0.000000
9,2010-01-05,XLB,34.080002,34.180000,33.810001,25.571341,8839200.0,1.0,0.001849,25.646683,25.413585,100.000000,66.666667,100.000000,25.530134,25.530134,19.350000,0.000000,0.000000
18,2010-01-06,XLB,34.139999,34.830002,34.080002,26.005896,8092000.0,2.0,0.015832,26.244230,25.133212,100.000000,100.000000,100.000000,25.688721,25.688721,19.160000,0.000000,0.000000
27,2010-01-07,XLB,34.540001,34.580002,34.160000,25.803606,5757300.0,3.0,0.014499,26.185337,25.249548,71.075439,49.778305,100.000000,25.717443,25.717443,19.059999,0.000000,0.000000
36,2010-01-08,XLB,34.360001,34.919998,34.279999,26.163240,4642400.0,4.0,0.027781,26.375094,25.238110,81.119227,90.453606,100.000000,25.806602,25.806602,18.129999,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38529,2021-09-24,XLB,80.820000,81.500000,80.629997,78.505173,3625900.0,4.0,-0.938540,84.518731,76.403601,44.937128,-121.155943,28.495853,80.906556,80.367391,17.750000,1.476685,79.093468
38556,2021-09-27,XLB,81.279999,82.160004,81.269997,79.115227,5216500.0,0.0,-0.865318,84.207570,76.339436,46.717026,-84.869768,19.946115,80.765377,80.356097,18.760000,12.354767,79.015203
38565,2021-09-28,XLB,81.500000,81.620003,80.440002,78.137184,8285300.0,1.0,-0.876109,83.899532,76.183018,44.339944,-108.798185,26.518655,80.605417,80.326573,23.250000,10.101659,79.009244
38574,2021-09-29,XLB,80.680000,80.860001,80.199997,77.817635,6969800.0,2.0,-0.900070,83.596839,76.013405,43.590287,-115.261796,28.337323,80.466622,80.311489,22.559999,4.564962,78.935362


In [300]:
insample_risk_indicator.vix.describe()

count    2957.000000
mean       18.105293
std         7.272476
min         9.140000
25%        13.370000
50%        16.209999
75%        20.629999
max        82.690002
Name: vix, dtype: float64

In [301]:
insample_risk_indicator.vix.quantile(0.996)

57.212001831054636

In [302]:
insample_risk_indicator.turbulence.describe()

count    2957.000000
mean        9.574998
std        12.733796
min         0.000000
25%         3.875439
50%         6.850858
75%        11.555260
max       258.292392
Name: turbulence, dtype: float64

In [303]:
insample_risk_indicator.turbulence.quantile(0.996)

88.2834426832541

In [304]:
trade

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence,prediction
0,2021-10-01,XLB,79.489998,80.800003,78.889999,77.827309,9250000.0,4.0,-0.978685,82.849750,75.685198,44.479083,-107.583836,37.463188,80.236743,80.251988,21.100000,5.313178,78.606171
0,2021-10-01,XLE,52.470001,53.889999,52.470001,50.451199,41656000.0,4.0,1.182468,50.653987,42.164670,59.586544,191.736139,29.265956,45.805905,45.741456,21.100000,5.313178,47.049043
0,2021-10-01,XLF,37.630001,38.410000,37.490002,37.005863,70444000.0,4.0,0.098725,37.574479,35.488708,53.517077,13.822004,5.497651,36.698644,36.249977,21.100000,5.313178,35.665288
0,2021-10-01,XLI,98.459999,99.709999,97.480003,96.721672,17051800.0,4.0,-0.976382,101.326237,95.073027,44.433481,-120.623472,39.320859,99.246636,99.847296,21.100000,5.313178,96.487770
0,2021-10-01,XLK,149.860001,151.899994,148.339996,149.482285,22741900.0,4.0,-1.163567,159.062870,147.038155,47.870406,-166.873679,47.408754,153.858114,152.269129,21.100000,5.313178,155.020169
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,2023-04-27,XLK,146.800003,149.320007,146.440002,149.210007,6042400.0,3.0,0.906210,151.760708,145.221293,57.138356,43.554032,10.029523,147.182253,143.204452,17.030001,4.575693,147.305152
394,2023-04-27,XLP,76.220001,77.110001,76.169998,77.029999,11287800.0,3.0,0.892918,77.176238,74.163761,61.967635,105.218450,34.078385,74.680582,73.532967,17.030001,4.575693,74.186167
394,2023-04-27,XLU,68.459999,69.139999,68.400002,69.080002,12466700.0,3.0,0.633085,70.845444,67.185557,53.290458,42.102205,4.528401,67.964842,67.168816,17.030001,4.575693,69.092036
394,2023-04-27,XLV,131.770004,132.589996,130.619995,132.399994,10008900.0,3.0,1.086855,136.802596,129.195405,52.366155,21.773054,4.270448,130.932522,130.043549,17.030001,4.575693,132.738247


In [317]:
e_trade_gym = StockTradingEnv(df = trade, turbulence_threshold = 70,risk_indicator_col='vix', **env_kwargs)

VIX threshold 70으로 지정함

In [316]:
trained_model = trained_a2c
env, _ = e_trade_gym.get_sb_env()  # Create the environment
df_account_value_a2c, df_actions_a2c = DRLAgent.DRL_prediction(
    model=trained_model, 
    environment=env)


AttributeError: 'DummyVecEnv' object has no attribute 'get_sb_env'

In [ ]:
df_account_value_a2c.shape

(399, 2)

## Back testing

In [ ]:
df_result_a2c = df_account_value_a2c.set_index(df_account_value_a2c.columns[0])
df_account_value_a2c.to_csv("df_account_value_a2c.csv")
#baseline stats
print("==============Get Baseline Stats===========")
df_dji_ = get_baseline(
        ticker="^DJI", 
        start = TRADE_START_DATE,
        end = TRADE_END_DATE)
stats = backtest_stats(df_dji_, value_col_name = 'close')
df_dji = pd.DataFrame()
df_dji['date'] = df_account_value_a2c['date']
df_dji['account_value'] = df_dji_['close'] / df_dji_['close'][0] * env_kwargs["initial_amount"]
df_dji.to_csv("df_dji.csv")
df_dji = df_dji.set_index(df_dji.columns[0])
df_dji.to_csv("df_dji+.csv")

result = pd.DataFrame(df_result_a2c)

"""result = pd.merge(df_result_a2c, df_result_ddpg, left_index=True, right_index=True)
result = pd.merge(result, df_result_td3, left_index=True, right_index=True)
result = pd.merge(result, df_result_ppo, left_index=True, right_index=True)
result = pd.merge(result, df_result_sac, left_index=True, right_index=True)
result = pd.merge(result, MVO_result, left_index=True, right_index=True)"""
result = pd.merge(result, df_dji, left_index=True, right_index=True)


result.columns = ['a2c','dji']

print("result: ", result)
result.to_csv("result.csv")

NameError: name 'df_account_value_a2c' is not defined

In [ ]:
%matplotlib inline
plt.rcParams["figure.figsize"] = (15,5)
plt.figure();
result.plot();

NameError: name 'result' is not defined

<Figure size 1500x500 with 0 Axes>

In [ ]:
df_result_a2c.iloc[-1,0]

NameError: name 'df_result_a2c' is not defined

In [ ]:
print('A2C로 얻은 투자 수익률>>', round((df_result_a2c.iloc[-1,0]/df_result_a2c.iloc[0,0]-1)*100,2),'%')

NameError: name 'df_result_a2c' is not defined

In [ ]:
df_daily_return = df_account_value_a2c

NameError: name 'df_account_value_a2c' is not defined

In [ ]:
df_actions_a2c

NameError: name 'df_actions_a2c' is not defined